In [ ]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt

# P1 Tarea 6


class Crank_Nicolson(object):
    def __init__(self, condiciones_borde, condiciones_iniciales):
        '''Inicializa la clase con los valores
        de las condiciones de borde, condiciones
        iniciales, mu, gamma, etc.'''
        self.cb = condiciones_borde  # n(x=0,t)=1 y n(x=1,t)=0
        self.ci = condiciones_iniciales  # n(x,0)=exp(-x**2/0.1)
        self.mu = 1.5
        self.gamma = 0.001
        self.t_actual = 0
        self.numinterx = 499
        self.numptosx = 500
        self.dx = 1/self.numinterx

    # matrices tridiagonales

    def calcula_d(self, n, dt):
        d = np.zeros(self.numptosx)
        r = dt*self.gamma/(2*(self.dx**2))
        for i in range(1, self.numptosx - 1):
            d[i] = r * n[i+1] + (1-2*r) * n[i] + r * n[i-1]
        return d

    def calcula_alpha_y_beta(self, n, dt):
        d = self.calcula_d(n, dt)
        alpha = np.zeros(self.numptosx)
        beta = np.zeros(self.numptosx)
        r = dt*self.gamma/(2*(self.dx**2))
        Aplus = -1 * r
        Acero = (1 + 2 * r)
        Aminus = -1 * r
        alpha[0] = 0
        beta[0] = 1  # viene de la condicion de borde n(t, 0) = 1, n(t, 1) = 0
        for i in range(1, self.numptosx - 1):
            alpha[i] = -Aplus / (Acero + Aminus*alpha[i-1])
            beta[i] = (d[i] - Aminus*beta[i-1]) / (Aminus*alpha[i-1] + Acero)
        return alpha, beta

    def avanza_CN(self, n, nnext, dt):
        # recibe arreglos n, nnext, alpha y beta
        '''Retorna el valor de la densidad de
        la especie para un paso de tiempo dt, por
        el metodo de Crank-Nicolson'''
        alpha, beta = self.calcula_alpha_y_beta(n, dt)
        nnext[0], nnext[499] = self.cb
        for i in range(1, self.numptosx - 1):
            nnext[i] = alpha[i] * nnext[i+1] + beta[i]
        return nnext

    def avanza_EulerP1(self, n, nnext, dt):
        '''Retorna el valor de la densidad de
        la especie para un paso de tiempo dt, por
        el metodo de Euler Explicito'''
        nnext[0], nnext[499] = self.cb
        for i in range(self.numptosx):
            nnext[i] = dt * (self.mu*n[i]-self.mu*(n[i]**2)) + n[i]
        return nnext

    def avanza_EulerP2(self, n, nnext, dt):
        '''Retorna el valor de la densidad de
        la especie para un paso de tiempo dt, por
        el metodo de Euler Explicito'''
        nnext[0], nnext[499] = self.cb
        for i in range(self.numptosx):
            nnext[i] = dt * (self.mu*n[i]-self.mu*(n[i]**3)) + n[i]
        return nnext


def condicion_inicial_y_borde(condiciones, numptosx, pregunta, arreglox=None):
    if pregunta == 1:
        ci = np.zeros(len(arreglox))
        for i in range(len(arreglox)):
            ci[i] = np.exp((-arreglox[i]**2)/0.1)
        ci[0], ci[1] = condiciones
    else:
        ci = np.random.uniform(low=-0.3, high=0.3, size=numptosx)
    return ci

# Main Setup P1


numptosx = 500
dt = 0.01
numptost = 5/dt + 1
xi = np.arange(0, 1, 1/(numptosx - 1))
ci = condicion_inicial_y_borde([1, 0], numptosx, 1, xi)  # n(0,x)
Fisher = Crank_Nicolson([1, 0], ci)


# solucion parte difusion


n = ci
nnext = np.zeros(numptosx)
N = np.zeros((numptosx, int(numptost)))
N[:, 0] = ci
for i in range(1, int(numptost)):
    nnext = Fisher.avanza_CN(n, nnext, dt)
    n = nnext
    N[:, i] = nnext

# solucion partes reaccion


n = ci
nnext = np.zeros(numptosx)
M = np.zeros((numptosx, int(numptost)))
M[:, 0] = ci
for j in range(1, int(numptost)):
    nnext = Fisher.avanza_EulerP1(n, nnext, dt)
    n = nnext
    M[:, j] = nnext
Ngen = N+M  # solucion general
Ngen[:, 0] = ci
x = np.linspace(0, 1, numptosx)
fig1 = plt.figure(1)
fig1.clf()
for j in range(len(N[0])):
    plt.plot(x, Ngen[:, j], 'r-')
plt.grid(True)
plt.show()


# P2 Tarea 6


# Main setup P2


numptosx = 500
np.random.seed(10)
ci = condicion_inicial_y_borde([0, 0], numptosx, 2)
dt = 0.01
numptost = 5/dt + 1


# solucion parte difusion


n = np.zeros(numptosx)
nnext = np.zeros(numptosx)
N = np.zeros((numptosx, int(numptost)))
N[:, 0] = ci
Fisher = Crank_Nicolson([0, 0], ci)
for i in range(1, int(numptost)):
    nnext = Fisher.avanza_CN(n, nnext, dt)
    n = nnext
    N[:, i] = nnext

# solucion partes reaccion


n = np.zeros(numptosx)
nnext = np.zeros(numptosx)
M = np.zeros((numptosx, int(numptost)))
M[:, 0] = ci
for j in range(1, int(numptost)):
    nnext = Fisher.avanza_EulerP2(n, nnext, dt)
    n = nnext
    M[:, j] = nnext
Ngen = N+M  # solucion general
Ngen[:, 0] = ci
x = np.linspace(0, 1, numptosx)
fig2 = plt.figure(2)
fig2.clf()
for j in range(len(N[0])):
    plt.plot(x, Ngen[:, 0], 'b-')
plt.grid(True)
plt.show()
